In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(
    api_key=os.getenv("GROQ_TOKEN"),
    base_url=os.getenv("BASE_URL_GROQ")
)

def get_response(prompt: str) -> str:
    response = client.chat.completions.create(
        model = os.getenv("GROQ_MODEL"),
        messages=[
            {'role':'user', 'content':prompt}
        ]
    )

    return response.choices[0].message.content

In [ ]:
## Часть 1
### Задача 1.1
transactions = [
    "Оплата Netflix подписка 990 руб",
    "Покупка продуктов в Пятёрочке 3450 руб",
    "Заправка на АЗС Shell 2100 руб",
    "Оплата коммунальных услуг 8500 руб",
    "Покупка книг на Ozon 1200 руб",
    "Ресторан 'Теремок' обед 850 руб"
]

zero_shot_prompt = f"""Классифицируй транзакции по категориям. Транзакции обрамленны тройными обратными кавычками. Категории для классификации:
- РАЗВЛЕЧЕНИЯ (подписки, кино, концерты)
- ПРОДУКТЫ (супермаркеты, продуктовые магазины)
- ТРАНСПОРТ (бензин, общественный транспорт, такси)
- ЖКХ (коммунальные услуги, интернет)
- ОБРАЗОВАНИЕ (книги, курсы)
- ПИТАНИЕ (рестораны, кафе)

```{'\n'.join(transactions)}```"""

one_shot_prompt = f"""Классифицируй транзакции по категориям. Транзакции обрамленны тройными обратными кавычками. Категории для классификации:
- РАЗВЛЕЧЕНИЯ (подписки, кино, концерты)
- ПРОДУКТЫ (супермаркеты, продуктовые магазины)
- ТРАНСПОРТ (бензин, общественный транспорт, такси)
- ЖКХ (коммунальные услуги, интернет)
- ОБРАЗОВАНИЕ (книги, курсы)
- ПИТАНИЕ (рестораны, кафе)

Примеры:
- Покупка продуктов -> ПРОДУКТЫ

```{'\n'.join(transactions)}```"""

few_shot_prompt = f"""Классифицируй транзакции по категориям. Транзакции обрамленны тройными обратными кавычками. Категории для классификации:
- РАЗВЛЕЧЕНИЯ (подписки, кино, концерты)
- ПРОДУКТЫ (супермаркеты, продуктовые магазины)
- ТРАНСПОРТ (бензин, общественный транспорт, такси)
- ЖКХ (коммунальные услуги, интернет)
- ОБРАЗОВАНИЕ (книги, курсы)
- ПИТАНИЕ (рестораны, кафе)

Примеры:
- Покупка продуктов 1000 рублей -> ПРОДУКТЫ
- Подписка на курс 500 рублей -> ОБРАЗОВАНИЕ
- Такси до работы 200 рублей -> ТРАНСПОРТ

```{'\n'.join(transactions)}```"""

print('==> ZERO-SHOT <==')
print(get_response(zero_shot_prompt))
print('==> ONE-SHOT <==')
print(get_response(one_shot_prompt))
print('==> FEW-SHOT <==')
print(get_response(few_shot_prompt))

==> ZERO-SHOT <==
<think>
Хорошо, мне нужно классифицировать эти транзакции по категориям. Давайте посмотрим на каждую из них по отдельности.

Первая транзакция: "Оплата Netflix подписка 990 руб". Netflix — это сервис потокового вещания, то есть подписка на контент, который можно смотреть, например, фильмы и сериалы. Это явно относится к развлечениям. Категория РАЗВЛЕЧЕНИЯ.

Вторая: "Покупка продуктов в Пятёрочке 3450 руб". Пятёрочка — это супермаркет, значит, покупка товаров первой необходимости, продуктов питания. Это ПРОДУКТЫ.

Третья: "Заправка на АЗС Shell 2100 руб". АЗС — автозаправочная станция, где покупают бензин для машины. Это относится к транспорту, так как связано с транспортными расходами. Категория ТРАНСПОРТ.

Четвёртая: "Оплата коммунальных услуг 8500 руб". Коммунальные услуги включают воду, свет, газ и интернет. Здесь упомянуто ЖКХ (жилищно-коммунальные хозяйства). Надо посмотреть, входят ли они в категорию ЖКХ. Да, ЖКХ — именно та категория, где перечислены коммунальн

In [ ]:
### Задача 1.2
response = client.chat.completions.create(
    model = os.getenv("GROQ_MODEL"),
    messages = [
        {
            'role':'system',
            'content':'''Классифицируй транзакции по категориям:
            - РАЗВЛЕЧЕНИЯ (подписки, кино, концерты)
            - ПРОДУКТЫ (супермаркеты, продуктовые магазины)
            - ТРАНСПОРТ (бензин, общественный транспорт, такси)
            - ЖКХ (коммунальные услуги, интернет)
            - ОБРАЗОВАНИЕ (книги, курсы)
            - ПИТАНИЕ (рестораны, кафе)'''
        },
        {'role':'user', 'content':'Покупка продуктов 1000 рублей'},
        {'role':'assistant','content':'ПРОДУКТЫ'},
        {'role':'user', 'content':'Подписка на курс 500 рублей'},
        {'role':'assistant','content':'ОБРАЗОВАНИЕ'},
        {'role':'user', 'content':'Такси до работы 200 рублей'},
        {'role':'assistant','content':'ТРАНСПОРТ'},
        {'role':'user','content':'Оплата коммунальных услуг 8500 руб'}
    ]
)

print(response.choices[0].message.content)

<think>
Хорошо, давайте разберемся с этой транзакцией. Оплата коммунальных услуг на 8500 рублей. Нужно определить, в какую категорию её отнести.

Сначала вспомню категории, которые были предложены. Категории: РАЗВЛЕЧЕНИЯ, ПРОДУКТЫ, ТРАНСПОРТ, ЖКХ, ОБРАЗОВАНИЕ, ПИТАНИЕ. 

Коммунальные услуги обычно включают плату за воду, электричество, газ, отопление, интернет. В списке категорий есть ЖКХ, которая как раз охватывает коммунальные услуги и интернет. 

Проверю предыдущие ответы: например, транспорт (такси) отнесли в ТРАНСПОРТ, подписку на курс — в ОБРАЗОВАНИЕ. Значит, коммунальные услуги должны соответствовать категории ЖКХ. 

Проверяю, не могла ли пользователь описать это как что-то другое, но тут всё однозначно: "оплата коммунальных услуг". Значит, правильно отнести в ЖКХ. 

Не вижу причин сомневаться, другие транзакции были однозначными, и здесь тоже нет противоречий. Ответ — ЖКХ.
</think>

ЖКХ


In [7]:
## Часть 2
### Задача 2.1
product_description = """
Смартфон Samsung Galaxy S23 Ultra 256GB. Экран 6.8 дюймов Dynamic AMOLED 2X,
разрешение 3088x1440. Процессор Snapdragon 8 Gen 2, 12GB RAM. Камеры: основная
200MP, ультрашироко угольная 12MP, телефото 10MP (3x) и 10MP (10x). Батарея 5000mAh
с быстрой зарядкой 45W. Цена: 89 990 рублей.
"""

simple_prompt = f"""Проанализируй описание смартфона обрамленного тройными обратными кавычками и дай рекомендацию о покупке

```{product_description}```"""

multi_step_prompt = f"""Проанализируй описание смартфона обрамленного тройными обратными кавычками и дай рекомендацию. Анализ проводи по следующим шагам:
1. Извлеки ключевые технические характеристики
2. Оцени соотношение цена/качество по 10-балльной шкале
3. Определи целевую аудиторию (для каких пользователей подходит)
4. Укажи главные преимущества (3 пункта)
5. Укажи возможные недостатки (2 пункта)
6. Дай финальную рекомендацию о покупке

```{product_description}```"""

print('==> SIMPLE PROMPT <==')
print(get_response(simple_prompt))
print('==> MULTI-STEP PROMPT <==')
print(get_response(multi_step_prompt))

==> SIMPLE PROMPT <==
<think>
Хорошо, давай разберёмся с этим запросом. Нужно проанализировать описание смартфона Samsung Galaxy S23 Ultra и дать рекомендацию о покупке. Сначала я должен понять, какие ключевые характеристики важно учитывать для такого устройства. 

Начнём с экрана: 6.8 дюймов Dynamic AMOLED 2X с разрешением 3088x1440. Это хороший экран, высокое разрешение, подходит для видео, игр, чтения. AMOLED означает яркое и контрастное изображение, правда, нужно учесть возможную утомляемость глаз при длительном просмотре.

Процессор Snapdragon 8 Gen 2 последний, значит, высокая производительность. Это важно для многооконности, игр, приложений. 12 ГБ ОЗУ — тоже достаточно, чтобы не было подкачки, но для пользователей, которые в основном делают что-то одно, может быть и не везде. 

Камеры: основная 200 МП — очень высокий мегапиксельный показатель, но важно, что на деле качество зависит не только от МП, но и от сенсора, обработки. Если у Samsung есть хорошие алгоритмы улучшения снимк

In [8]:
## Часть 3
### Задача 3.1
question = """В компании работает 50 сотрудников. 60% из них используют MacBook, остальные — Windows.
Из тех, кто использует MacBook, 40% работают в отделе разработки. Из тех, кто использует
Windows, 25% работают в отделе разработки. Сколько разработчиков работает в компании?"""

simple_prompt = f"""Реши задачу обрамленную в тройные обратные кавычки. Дай только ответ

```{question}```"""

chain_of_thought = f"""Реши задачу обрамленную тройными обратными кавычками по шагово.

```{question}```"""

print('==> SIMPLE PROMPT <==')
print(get_response(simple_prompt))
print('==> CHAIN OF THOUGHT PROMPT <==')
print(get_response(chain_of_thought))

==> SIMPLE PROMPT <==
<think>
Хорошо, давайте разберемся с этой задачей по шагам. Сначала нужно понять, сколько сотрудников используют MacBook и сколько — Windows. Всего в компании 50 человек. 60% из них — это MacBook пользователи, значит, сначала посчитаю 60% от 50. 

60% от 50 — это 0.6 * 50 = 30 человек. Значит, MacBook используют 30 сотрудников, а остальные 50 - 30 = 20 человек используют Windows.

Теперь нужно определить, сколько из этих групп работают в отделе разработки. Для MacBook пользователей это 40%. Посчитаю 40% от 30: 0.4 * 30 = 12 человек.

У тех, кто на Windows, 25% в разработке. Считаем 25% от 20: 0.25 * 20 = 5 человек.

Теперь сложим разработчиков из обеих групп: 12 + 5 = 17. 

Получается, в компании работает 17 разработчиков.
</think>

Ответ: 17
==> CHAIN OF THOUGHT PROMPT <==
<think>
Хорошо, давайте разберемся с этой задачей шаг за шагом. В компании 50 сотрудников. Сначала нужно определить, сколько из них используют MacBook, а сколько — Windows. Указано, что 60% исп

In [9]:
### Задача 3.2
question = '''В офисе 80 сотрудников. 45% предпочитают кофе, 35% — чай, остальные — воду.
Из любителей кофе 60% добавляют сахар. Из любителей чая 40% добавляют сахар.
Сколько человек в офисе пьют напитки с сахаром?'''

prompt = f'''Реши задачу обрамленную в тройные обратные кавычки. Для обучения используй следующие пример:

Задача: В школе 100 учеников. 30% из них занимаются футболом, 20% — баскетболом,
остальные не занимаются спортом. Сколько учеников не занимается спортом?

Решение шаг за шагом:
Шаг 1: Ученики, занимающиеся футболом: 100 * 0.3 = 30
Шаг 2: Ученики, занимающиеся баскетболом: 100 * 0.2 = 20
Шаг 3: Общее количество спортсменов: 30 + 20 = 50
Шаг 4: Не занимаются спортом: 100 - 50 = 50
Ответ: 50 учеников

```{question}```
'''

print(get_response(prompt))

<think>
Хорошо, давайте разберемся с этой задачей шаг за шагом. Сначала в офисе 80 сотрудников. Нам нужно найти, сколько человек пьют напитки с сахаром, учитывая, что из любителей кофе (45%) 60% добавляют сахар, а из любителей чая (35%) — 40% добавляют сахар.

Первым делом, наверное, нужно посчитать количество людей, которые предпочитают кофе и чай. 

Начнем с кофелюбов. 45% от 80 сотрудников: 80 * 0,45. Считаю: 80 * 0,45 = 36. Значит, 36 человек пьют кофе. Из них 60% добавляют сахар. Тогда количество кофелюбов с сахаром: 36 * 0,6. Посчитаю: 36 * 0,6 = 21,6. Ой, но людей не может быть дробное число. Возможно, я где-то ошибся? Нет, в условии не сказано, что нужно округлять, так что, наверное, дальше будет 21,6.

Теперь чайные. 35% от 80: 80 * 0,35 = 28. 28 человек предпочитают чай. Из них 40% добавляют сахар. Тогда 28 * 0,4 = 11,2. Опять дробное число. Хотя, возможно, в реальности это должно быть целое, но в условии, видимо, можно продолжать с десятичными.

Теперь суммируем тех, кто пье

In [14]:
## Часть 4
### Задача 4.1
question = '''На парковке было 15 машин. Утром приехало в 2 раза больше машин, чем было.
К обеду половина всех машин уехала. Потом приехало еще 8 машин.
Сколько машин на парковке сейчас?'''

prompt = f"""Реши задачу обрамленную тройными обратными кавычками. Для решения используй трех независимых экспертов которые думают и решают задачу различными путями. Окончательное значение решения будет тем которое получили большинство экспертов.

```{question}```"""

for i in range(3):
    print(f'==> ПОПЫТКА {i} <==')
    print(get_response(prompt))

==> ПОПЫТКА 0 <==
<think>
Хорошо, давайте разберемся с этой задачей. Начнем с того, что в парке изначально было 15 машин. Утром приехало в 2 раза больше, чем было. То есть 15 умножить на 2 получается 30. Теперь к обеду половина всех машин уехала. Нужно понять, сколько всего машин стало до того, как уехали. Изначально было 15, приехало 30, значит, после того как приехали утром, получилось 45 машин. Половина из них уехала, то есть 45 делить на 2 это 22.5, но машину нельзя разделить на пол, поэтому, наверное, 22 или 23, но в примерах таких задач обычно берут целое число. Возможно, я где-то ошибся.

Потом приехало еще 8. Если после обеда осталось 22.5 машин, то прибавляем 8, получается 30.5. Но это не целое число. Значит, где-то неправильно посчитал. Допустим, после утреннего приезда 15+30=45 машин. Половина уехала, то есть осталось 45/2=22.5. Но это невозможно. Возможно, в условии задачи ошибка или я неправильно понял. Может быть, половина от всех машин, которые были утром? То есть после 

=== ИТОГОВЫЙ АНАЛИЗ ===

Консенсусный ответ встретился: 3 раза из 3

Стабильность результата: Высокая

In [3]:
## Задача 5
### Часть 5.1
code = """
def process_data(data):
    result = []
    for item in data:
        if item > 0:
            result.append(item * 2)
    return result
"""

iteration_1 = f"""Проанализируй код в тройных обратных кавычках в одно предложение

```{code}```"""

iteration_2 = f"""Проанализируй код в тройных обратных кавычках. Определи следующие параметры:
- Описание: одно предложение
- Язык программирования: язык программирования на которой написан код
- Сложность: оцени сложность по 10-ти бальной шкале

```{code}```"""

iteration_3 = f"""Проанализируй код в тройных обратных кавычках. Определи следующие параметры:
- Описание: одно предложение
- Язык программирования: язык программирования на которой написан код
- Сложность: оцени сложность по 10-ти бальной шкале
- Эффективность: оцени эффективность работы кода по 10-ти бальной шкале
- Улучшения: сгенерируй предложения по улучшению, списком не более 3 пунктов

```{code}```"""

iteration_4 = f"""Проанализируй код в тройных обратных кавычках. Определи следующие параметры:
- Описание: одно предложение
- Язык программирования: язык программирования на которой написан код
- Сложность: оцени сложность по 10-ти бальной шкале
- Эффективность: оцени эффективность работы кода по 10-ти бальной шкале
- Улучшения: сгенерируй предложения по улучшению, списком не более 3 пунктов
- Использование: приведи 2 примера где этот код можно использовать
- Баги: сгенерируй список из не более пунктов потенциальных багов

```{code}```"""

iterations = [iteration_1, iteration_2, iteration_3, iteration_4]

for i, prompt in enumerate(iterations):
    print(f'==> {i} ITERATION <==')
    print(f'PROMPT ==>\n{prompt}')
    print('===> RESPONSE <===')
    print(get_response(prompt))

==> 0 ITERATION <==
PROMPT ==>
Проанализируй код в тройных обратных кавычках в одно предложение

```
def process_data(data):
    result = []
    for item in data:
        if item > 0:
            result.append(item * 2)
    return result
```
===> RESPONSE <===
<think>
Хорошо, давайте разберёмся с этим кодом. Функция называется process_data и принимает параметр data. Она создаёт пустой список result. Затем начинается цикл for, который перебирает каждый элемент в data. Внутри цикла есть условие: если элемент больше нуля, тогда он умножается на два и добавляется в результат. В конце возвращается результирующий список.

Нужно понять, что делает эта функция в целом. Похоже, что она фильтрует data, оставляя только положительные числа, и удваивает их. То есть входные данные — список, а результат — список удвоенных положительных чисел.

Теперь проверю, нет ли ошибок. Цикл for правильно перебирает элементы. Условие item > 0 — нормально, но что, если в data есть нули или отрицательные числа? Они

In [ ]:
### Задача 5.2
examples = """
Отзыв: Отличный продукт, рекомендую! → Позитивный
Отзыв: Полный провал, не покупайте → Негативный
Отзыв: Обычный товар, ничего особенного → Нейтральный
"""

test_review = "Ну просто великолепно, опять сломалось через неделю!"

simple_few_shot_prompt = f"""Классифицируй отзывы обрамленные тройными обратными кавычками.
Отзыв: Отличный продукт, рекомендую! → Позитивный
Отзыв: Полный провал, не покупайте → Негативный
Отзыв: Обычный товар, ничего особенного → Нейтральный

Отзыв:```{test_review}```"""

few_shot_prompt_with_sarcasm = f"""Классифицируй отзывы обрамленные тройными обратными кавычками.
Отзыв: Отличный продукт, рекомендую! → Позитивный
Отзыв: Полный провал, не покупайте → Негативный
Отзыв: Обычный товар, ничего особенного → Нейтральный
Отзыв: Товар супер, даже не включается → Негативный

Отзыв:```{test_review}```"""

few_shot_prompt_with_sarcasm_instruction = f"""Классифицируй отзывы обрамленные тройными обратными кавычками. Если отзыв содержит сарказм классифицируй его как "Сарказм"
Отзыв: Отличный продукт, рекомендую! → Позитивный
Отзыв: Полный провал, не покупайте → Негативный
Отзыв: Обычный товар, ничего особенного → Нейтральный
Отзыв: Товар супер, даже не включается → Сарказм

Отзыв:```{test_review}```"""

few_shot_prompt_with_sarcasm_description = f"""Классифицируй отзывы обрамленные тройными обратными кавычками. Если отзыв содержит сарказм классифицируй его как "Сарказм". Если ты классифицируешь отзыв в класс "Сарказм" объясни причину.
Отзыв: Отличный продукт, рекомендую! → Позитивный
Отзыв: Полный провал, не покупайте → Негативный
Отзыв: Обычный товар, ничего особенного → Нейтральный
Отзыв: Товар супер, даже не включается → Сарказм

Отзыв:```{test_review}```"""

prompts = {
    'simple_few_shot_prompt': simple_few_shot_prompt, 'few_shot_prompt_with_sarcasm': few_shot_prompt_with_sarcasm, 
    'few_shot_prompt_with_sarcasm_instruction': few_shot_prompt_with_sarcasm_instruction, 
    'few_shot_prompt_with_sarcasm_description': few_shot_prompt_with_sarcasm_description
    }

for i, prompt in enumerate(prompts):
    print(f'=== ITERATION {i}: {prompt} ===')
    print(get_response(prompts[prompt]))

=== ITERATION 0: simple_few_shot_prompt
<think>
Хорошо, давай разберемся с этим запросом. Пользователь попросил классифицировать отзыв, который находится между тройными обратными кавычками. Сначала я должен понять формат входных данных. В примере показаны три отзыва с их классификацией: позитивный, негативный и нейтральный. Теперь нужно обработать новый отзыв, который заключен в тройные обратные кавычки.

В отзыве написано: "Ну просто великолепно, опять сломалось через неделю!". Сначала я должен прочитать и понять тон этого отзыва. Хотя первая часть содержит слово "великолепно", которое обычно положительное, но затем следует "опять сломалось через неделю", что явно негативно. Это противоречие может быть сарказмом. Часто люди используют сарказм, чтобы выразить недовольство, начав с какого-то позитива, но подчеркнув проблему.

Нужно определить, как это классифицировать. Сарказм часто относится к негативным отзывам, так как основная мысль — критика. В примерах, которые предоставил пользов

In [6]:
## Часть 6
### Задача 6.1
resume = """
Иван Смирнов, 28 лет
Образование: Бакалавр, МГУ, Прикладная математика (2014-2018)
Опыт работы:
- Junior Python Developer, StartupXYZ (2018-2019): разработка веб-приложений на Flask
- Middle Python Developer, TechCompany (2019-2021): backend разработка, Django, PostgreSQL
- Senior Python Developer, BigCorp (2021-настоящее время): архитектура микросервисов,
  FastAPI, Docker, Kubernetes, управление командой из 3 человек

Навыки: Python, Django, FastAPI, PostgreSQL, Docker, Kubernetes, Git, CI/CD
Языки: Русский (родной), Английский (B2)
"""

vacancy = """
Вакансия: Tech Lead Python
Требования:
- 5+ лет опыта разработки на Python
- Опыт проектирования архитектуры
- Опыт управления командой
- Знание Docker и Kubernetes
- Английский от B2
Зарплата: 300-400k
"""

example = """
Кандидат: Мария Иванова, Junior Developer, 2 года опыта
Вакансия: Senior Developer, требуется 5+ лет

Анализ:
Шаг 1: Опыт - 2 года
Шаг 2: Требуется 5+ лет, есть 2 года - НЕ соответствует
Шаг 3: Оценка - 0% по опыту
Шаг 4: Рекомендация - ОТКЛОНИТЬ (недостаточно опыта)
"""

prompt = f'''
Ты элитный сотрудник отдела HR в крупной IT компании. 

Ты ищешь кандидата на вакансию с описанием:
{vacancy}

Проанализируй резюме кандидата обрамленного в тройные обратные кавычки. Анализ проводи по следующим шагам:
1. Извлечь ключевые данные из резюме (опыт, навыки, образование)
2. Сопоставить с требованиями вакансии
3. Оценить каждое требование (соответствует/не соответствует/частично)
4. Рассчитать общий процент соответствия
5. Выявить сильные стороны кандидата
6. Выявить слабые стороны или пробелы
7. Дать финальную рекомендацию (пригласить/отклонить/запросить доп. инфо)

Пример анализа резюме:
{example}

```{resume}```
'''

print(f"""
=== FEW-SHOT EXAMPLE ===
{example}

=== АНАЛИЗ КАНДИДАТА ===
{get_response(prompt)}
""")


=== FEW-SHOT EXAMPLE ===

Кандидат: Мария Иванова, Junior Developer, 2 года опыта
Вакансия: Senior Developer, требуется 5+ лет

Анализ:
Шаг 1: Опыт - 2 года
Шаг 2: Требуется 5+ лет, есть 2 года - НЕ соответствует
Шаг 3: Оценка - 0% по опыту
Шаг 4: Рекомендация - ОТКЛОНИТЬ (недостаточно опыта)


=== АНАЛИЗ КАНДИДАТА ===
<think>
Хорошо, я должен проанализировать резюме Ивана Смирнова на должность Tech Lead Python. Сначала посмотрю на опыт. В резюме указано, что он 3 года работал Middle и Senior Python Developer. Всего 6 лет опыта? Давайте посчитаем: 2018-2019 (1 год как Junior), 2019-2021 (2 года Middle), 2021-сейчас (уже 3 года). В сумме это 6 лет, что соответствует требованию в 5+ лет.

Теперь проектирование архитектуры. В Senior позиции он архитектурировал микросервисы, значит, это есть. Управление командой: упомянуто, что ведет команду из 3 человек. Docker и Kubernetes в навыках, в Senior позиции указаны. Английский B2 — есть. 

Сильные стороны: опыт соответствует, архитектура, упра

In [7]:
## Часть 7
### Задача 7.1

news = [
    "Apple представила новый iPhone 15 с улучшенной камерой и процессором A17",
    "Центробанк повысил ключевую ставку до 16% на фоне роста инфляции",
    "Российские хоккеисты вышли в финал чемпионата мира",
    "Ученые обнаружили новую экзопланету в потенциально обитаемой зоне",
    "Правительство утвердило новый бюджет на следующий год",
    "Tesla начала производство грузовиков Cybertruck"
]

zero_shot = f'''Классифицируй новости обрамленные в тройные обратные кавычки по категориям:
- ТЕХНОЛОГИИ
- СПОРТ
- ПОЛИТИКА
- ЭКОНОМИКА
- НАУКА

```{'\n'.join(news)}```'''

few_shot_3 = f'''Классифицируй новости обрамленные в тройные обратные кавычки по категориям:
- ТЕХНОЛОГИИ
- СПОРТ
- ПОЛИТИКА
- ЭКОНОМИКА
- НАУКА

Примеры:
- Сегодня вышел новый iphone 20 -> Технологии
- В предверии финансового кризиса цены на нефть начали стремительно рости -> ЭКОНОМИКА
- Ученые из Палестины открыли новый вид пиницилина -> НАУКА

```{'\n'.join(news)}```'''

few_shot_3_inst = f'''Классифицируй новости обрамленные в тройные обратные кавычки по категориям:
- ТЕХНОЛОГИИ
- СПОРТ
- ПОЛИТИКА
- ЭКОНОМИКА
- НАУКА

Примеры:
- Сегодня вышел новый iphone 20 -> Технологии
- В предверии финансового кризиса цены на нефть начали стремительно рости -> ЭКОНОМИКА
- Ученые из Палестины открыли новый вид пиницилина -> НАУКА

Если новость может относится к нескольким категориям сразу определяй класс по наибольшей сфере влияния.

```{'\n'.join(news)}```'''

few_shot_3_cot = f'''Классифицируй новости обрамленные в тройные обратные кавычки по категориям:
- ТЕХНОЛОГИИ
- СПОРТ
- ПОЛИТИКА
- ЭКОНОМИКА
- НАУКА

Примеры:
- Сегодня вышел новый iphone 20 -> Технологии
- В предверии финансового кризиса цены на нефть начали стремительно рости -> ЭКОНОМИКА
- Ученые из Палестины открыли новый вид пиницилина -> НАУКА

Если новость может относится к нескольким категориям сразу определяй класс по наибольшей сфере влияния, также объясни свой выбор в пошаговом размышлении.

```{'\n'.join(news)}```'''

few_shot_3_sc = f'''Классифицируй новости обрамленные в тройные обратные кавычки по категориям:
- ТЕХНОЛОГИИ
- СПОРТ
- ПОЛИТИКА
- ЭКОНОМИКА
- НАУКА

Примеры:
- Сегодня вышел новый iphone 20 -> Технологии
- В предверии финансового кризиса цены на нефть начали стремительно рости -> ЭКОНОМИКА
- Ученые из Палестины открыли новый вид пиницилина -> НАУКА

Если новость может относится к нескольким категориям сразу представь трех независимых экспертов которые анализирует новости и дают классификацию по собственным размышлениям и подходам, итоговый класс будет по большинству совпадений между экспертами.

```{'\n'.join(news)}```'''

prompt_dict = {
    'Zero shot prompt': zero_shot,
    'Few shot prompt with 3 examples': few_shot_3,
    'Few shot prompt with instruction': few_shot_3_inst,
    'Few shot prompt with chain of thought': few_shot_3_cot,
    'Few shot prompt with self-consistency': few_shot_3_sc
}

for i, prompt_name in enumerate(prompt_dict):
    print(f'=== ITERATION {i}: {prompt_name} ===')
    print(get_response(prompt_dict[prompt_name]))

=== ITERATION 0: Zero shot prompt ===
<think>
Хорошо, мне нужно классифицировать каждую из шести новостных строк по категориям: ТЕХНОЛОГИИ, СПОРТ, ПОЛИТИКА, ЭКОНОМИКА, НАУКА. Начну с первой строки.

Первая строка: "Apple представила новый iPhone 15 с улучшенной камерой и процессором A17". Apple — компания, выпуск смартфонов, упоминается о новой модели и технических улучшениях. Это явно технологии.

Вторая: "Центробанк повысил ключевую ставку до 16% на фоне роста инфляции". Центробанк связан с денежной политикой, повышение ставки влияет на экономику. Категория — экономика.

Третья: "Российские хоккеисты вышли в финал чемпионата мира". Спорт, хоккей, международное соревнование. Беру в категорию СПОРТ.

Четвёртая: "Ученые обнаружили новую экзопланету в потенциально обитаемой зоне". Учёные, астрономическое открытие. Это наука.

Пятая: "Правительство утвердило новый бюджет на следующий год". Правительство принимает бюджетные решения, это политика или экономика? Но бюджет — это экономическая